In [105]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from numpy import array
from keras.models import load_model
import pandas as pd
import json
import numpy as np
  

In [82]:
# return training data
def get_train():
    seq = [[0.0, 0.1], [0.2, 0.3], [0.4, 0.5], [0.6, -0.7], [0.8, -0.9]]
    X = array(seq)
    y= array([1,1,1,0,0])
    y = array([[1,0],[1,0],[1,0],[0,1],[0,1]])
#     X, y = seq[:, 0], seq[:, 1]
    X = X.reshape((len(X), 2, 1))
    return X, y
 


In [85]:
# define model
model = Sequential()
model.add(LSTM(10, input_shape=(2,1)))
model.add(Dense(2, activation='sigmoid'))
# compile model
model.compile(loss='binary_crossentropy', optimizer='adam')
# fit model
X,y = get_train()


In [86]:
model.fit(X, y, epochs=300, shuffle=False, verbose=0)
# save model to single file
# model.save('lstm_model.h5')

In [89]:
ll =array([1,1.1]).reshape(1,2,1)
ll

array([[[ 1. ],
        [ 1.1]]])

In [90]:
# load model from single file
# model = load_model('lstm_model.h5')
# make predictions
yhat = model.predict(ll, verbose=0)
print(yhat)

[[ 0.83893681  0.17341484]]


In [92]:
data = json.load(open('motionpredict-export (1).json'))

In [458]:
df =pd.DataFrame()
y = []
x=[]

for key in data.keys():
    instance = data[key][:20]
    arr = []
    for sample in instance:
        arr.append([sample['accelerationX'], sample['accelerationY'], sample['accelerationZ']])
    x.append(arr)
    y.append(instance[0]['button'])
    df=pd.concat([df,pd.DataFrame(arr).transpose()])
y = pd.get_dummies(y)
y= y.as_matrix()
trainx = x[:-2]
testx = x[-2:]
trainy = y[:-2]
testy = y[-2:]

In [405]:
np.shape(y)

(24, 2)

In [233]:
model = Sequential()
model.add(LSTM(10, input_shape=(20,3)))
model.add(Dense(2, activation='sigmoid'))
# compile model
model.compile(loss='binary_crossentropy', optimizer='adam')
# fit model
model.fit(trainx, trainy, epochs=300, shuffle=False, verbose=0)



In [234]:
yhat = model.predict(array(testx).reshape(2,20,3), verbose=0)
yhat

array([[ 0.01213605,  0.98167449],
       [ 0.91811264,  0.12402326]], dtype=float32)

In [455]:
np.shape(x)
np.shape(trainx)
testy

array([[0, 1],
       [1, 0]], dtype=uint8)

In [467]:
np.shape(x)



(24, 20, 3)

In [459]:
all = []
for index, excersice in enumerate(x):
    for iteration in excersice:
        all.append([iteration,y[index]])
        


In [499]:
yy = array(all)[:,1]
xx = array(array(array(all)[:,0]).tolist())
xx = xx.reshape(len(xx),1,3)
yy = array(array(yy.tolist()))

In [550]:
model = Sequential()
# model.add(LSTM(10, input_shape=(1,3)))
model.add(LSTM(10, batch_input_shape=(1, xx.shape[1], xx.shape[2]), stateful=True))
model.add(Dense(2, activation='sigmoid'))
# compile model
model.compile(loss='binary_crossentropy', optimizer='adam')

# model.compile(loss='binary_crossentropy', optimizer='adam')
# fit model
n_epoch = 300
for i in range(n_epoch):
    model.fit(xx, yy, epochs=1, batch_size=1, verbose=3, shuffle=False)
    print("trained epoch", i)
    model.reset_states()

Epoch 1/1
trained epoch 0
Epoch 1/1
trained epoch 1
Epoch 1/1
trained epoch 2
Epoch 1/1
trained epoch 3
Epoch 1/1
trained epoch 4
Epoch 1/1
trained epoch 5
Epoch 1/1
trained epoch 6
Epoch 1/1
trained epoch 7
Epoch 1/1
trained epoch 8
Epoch 1/1
trained epoch 9
Epoch 1/1
trained epoch 10
Epoch 1/1
trained epoch 11
Epoch 1/1
trained epoch 12
Epoch 1/1
trained epoch 13
Epoch 1/1
trained epoch 14
Epoch 1/1
trained epoch 15
Epoch 1/1
trained epoch 16
Epoch 1/1
trained epoch 17
Epoch 1/1
trained epoch 18
Epoch 1/1
trained epoch 19
Epoch 1/1
trained epoch 20
Epoch 1/1
trained epoch 21
Epoch 1/1
trained epoch 22
Epoch 1/1
trained epoch 23
Epoch 1/1
trained epoch 24
Epoch 1/1
trained epoch 25
Epoch 1/1
trained epoch 26
Epoch 1/1
trained epoch 27
Epoch 1/1
trained epoch 28
Epoch 1/1
trained epoch 29
Epoch 1/1
trained epoch 30
Epoch 1/1
trained epoch 31
Epoch 1/1
trained epoch 32
Epoch 1/1
trained epoch 33
Epoch 1/1
trained epoch 34
Epoch 1/1
trained epoch 35
Epoch 1/1
trained epoch 36
Epoch 1/1
t

trained epoch 297
Epoch 1/1
trained epoch 298
Epoch 1/1
trained epoch 299


In [551]:
yhat = model.predict(xx, batch_size=1)
modelpred = []
actualpred = []
for val in yhat:
    modelpred.append(val.argmax())
for val in yy:
    actualpred.append(val.argmax())

In [549]:
results = pd.DataFrame(array(modelpred) == array(actualpred))
results[0].value_counts() / len(results[0])

True     0.579167
False    0.420833
Name: 0, dtype: float64

In [552]:
results = pd.DataFrame(array(modelpred) == array(actualpred))
results[0].value_counts() / len(results[0])

True     0.85625
False    0.14375
Name: 0, dtype: float64